In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
import spacy
from nltk.tokenize import word_tokenize
import nltk
from nltk import pos_tag

In [3]:
import time
import pandas as pd
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from nltk.sentiment import SentimentIntensityAnalyzer

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.cluster import KMeans

In [5]:
lemmatizer = WordNetLemmatizer()
# Function to clean text: Tokenization, Lemmatization, and Stopword removal
def clean_text(text):
    if isinstance(text, str):  # Check if the text is a string
        text = text.lower()  # Convert to lowercase
        tokens = word_tokenize(text)  # Tokenize the text

        # Lemmatize each token
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

        # Additional preprocessing (like removing stopwords)
        stop_words = set(stopwords.words('english'))
        tokens_without_stopwords = [token for token in lemmatized_tokens if token not in stop_words]

        # Rejoin tokens into a single string before passing to TextBlob
        cleaned_text = " ".join(tokens_without_stopwords)
        return cleaned_text
    else:
        # If the text is not a string (e.g., NaN or float), return an empty string or a default value
        return ""  # Or return a string like 'Invalid Text' if needed

In [6]:
nlp = spacy.load("en_core_web_lg")

def extract_entities(text):
    # Step 1: Use spaCy to extract entities (like companies, products, etc.)
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents if ent.label_ == "ORG"]  # Extracting organizations

    # Step 2: Apply POS tagging to the tokens in the text
    tokens = word_tokenize(text)  # Tokenize the text
    pos_tags = pos_tag(tokens)  # Get POS tags for the tokens

    # Step 3: Extract proper nouns (NNP) or important nouns (NN) using POS tagging
    nouns = [word for word, tag in pos_tags if tag in ["NNP","NN"]]  # Extracting proper nouns and common nouns

    # Combine NER and POS results
    refined_entities = list(set(entities + nouns))  # Combine and remove duplicates
    return refined_entities

In [7]:
  # Function to load data from Excel
def load_data_from_excel(file_path):
    # Read the Excel file
    df = pd.read_csv(file_path)
    
    # Assuming the Excel file has columns 'text' for the comments
    texts = df['cleaned_body'].tolist()  # List of texts (Reddit posts)
    return df, texts

In [8]:
# Function to perform sentiment analysis with TextBlob
def analyze_sentiment_with_textblob(texts):
    subjectivity = []
    polarity = []
    for text in texts:
        blob = TextBlob(text)
        subjectivity.append(blob.sentiment.subjectivity)
        polarity.append(blob.sentiment.polarity)
    return subjectivity, polarity


In [14]:
# Load data from Excel sheet
file_path = "/Users/jaredog/Downloads/cleaned_stock_data (1).csv"  # Replace with your actual Excel file path
df, texts = load_data_from_excel(file_path)

# --- 1. Preprocess the text ---
start_time = time.time()
cleaned_texts = [clean_text(text) for text in texts]
preprocessing_time = time.time() - start_time
print(f"Preprocessing Time: {preprocessing_time:.4f} seconds")


Preprocessing Time: 1.7118 seconds


In [15]:
# --- 2. Sentiment Analysis with TextBlob ---
start_time = time.time()
subjectivity, polarity = analyze_sentiment_with_textblob(cleaned_texts)
sentiment_analysis_time = time.time() - start_time
print(f"Sentiment Analysis Time: {sentiment_analysis_time:.4f} seconds")

# Append the sentiment analysis results to the dataframe
df['subjectivity'] = subjectivity
df['polarity'] = polarity

# Determine the overall sentiment (positive, negative, neutral)
sentiment_labels = []
for polarity_score in polarity:
    if polarity_score > 0:
        sentiment_labels.append('positive')
    elif polarity_score < 0:
        sentiment_labels.append('negative')
    else:
        sentiment_labels.append('neutral')

df['sentiment'] = sentiment_labels

Sentiment Analysis Time: 0.8853 seconds


In [13]:
# --- 3. Remove neutral sentiment rows ---
#df = df[df['sentiment'] != 'neutral']  # Drop all rows where sentiment is 'neutral'

df['cleaned_body'].tolist()
df.head()

,type,datetime,post_id,subreddit,title,author,url,upvotes,downvotes,upvote_ratio,body,cleaned_body,subjectivity,polarity,sentiment
0,post,2025-03-01 10:00:42,1j0w73o,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,AutoModerator,https://www.reddit.com/r/stocks/comments/1j0w7...,79,4.0,0.95,Please use this thread to discuss your portfol...,Please use this thread to discuss your portfol...,0.36803,0.176667,positive
1,comment,2025-03-04 13:54:48,mfykqf0,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,Jimlad73,https://www.reddit.com/r/stocks/comments/1j0w7...,12,NaN,NaN,100% S&P and bricking it,100% S&P and bricking it,0.00000,0.000000,neutral
2,comment,2025-03-11 00:18:18,mh4b63d,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,inopia,https://www.reddit.com/r/stocks/comments/1j0w7...,11,NaN,NaN,"100% 6-month tbills, lol :)","100% 6-month tbills, lol :)",0.85000,0.650000,positive
3,comment,2025-03-11 21:38:43,mha0zp1,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,thenuttyhazlenut,https://www.reddit.com/r/stocks/comments/1j0w7...,6,NaN,NaN,|Ticker|Company|Allocation|\n|:-|:-|:-|\n|ACGL...,|Ticker|Company|Allocation|\n|:-|:-|:-|\n|ACGL...,0.49250,0.267500,positive
4,comment,2025-03-13 15:57:56,mhl2a2j,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,elgrandorado,https://www.reddit.com/r/stocks/comments/1j0w7...,7,NaN,NaN,Ticker & %\n\nSPGI: 18.2%\n\nASML: 15.2%\n\nGO...,Ticker & %\n\nSPGI: 18.2%\n\nASML: 15.2%\n\nGO...,0.00000,0.000000,neutral


In [16]:
# --- 4. Named Entity Recognition ---
start_time = time.time()
# Apply NER only to the remaining (non-neutral) rows
# make sure no NaNs, and everything is a str
df['cleaned_body'] = df['cleaned_body'].fillna("").astype(str) #added
remaining_texts = df['cleaned_body'].tolist()  # Only the rows with non-neutral sentiment
entities = [extract_entities(text) for text in remaining_texts]
entity_extraction_time = time.time() - start_time
print(f"Entity Extraction Time: {entity_extraction_time:.4f} seconds")

# Append extracted entities to the dataframe
df['entities'] = entities
df.head()

Entity Extraction Time: 69.4438 seconds


,type,datetime,post_id,subreddit,title,author,url,upvotes,downvotes,upvote_ratio,body,cleaned_body,subjectivity,polarity,sentiment,entities
0,post,2025-03-01 10:00:42,1j0w73o,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,AutoModerator,https://www.reddit.com/r/stocks/comments/1j0w7...,79,4.0,0.95,Please use this thread to discuss your portfol...,Please use this thread to discuss your portfol...,0.36803,0.176667,positive,"[Losing, paper, portfolio, %, /, book, learn, ..."
1,comment,2025-03-04 13:54:48,mfykqf0,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,Jimlad73,https://www.reddit.com/r/stocks/comments/1j0w7...,12,NaN,NaN,100% S&P and bricking it,100% S&P and bricking it,0.00000,0.000000,neutral,"[S, P, %, S&P]"
2,comment,2025-03-11 00:18:18,mh4b63d,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,inopia,https://www.reddit.com/r/stocks/comments/1j0w7...,11,NaN,NaN,"100% 6-month tbills, lol :)","100% 6-month tbills, lol :)",0.85000,0.650000,positive,"[%, lol]"
3,comment,2025-03-11 21:38:43,mha0zp1,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,thenuttyhazlenut,https://www.reddit.com/r/stocks/comments/1j0w7...,6,NaN,NaN,|Ticker|Company|Allocation|\n|:-|:-|:-|\n|ACGL...,|Ticker|Company|Allocation|\n|:-|:-|:-|\n|ACGL...,0.49250,0.267500,positive,"[Design|12.75, |PBR|Petroleo, |WISE, insurance..."
4,comment,2025-03-13 15:57:56,mhl2a2j,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,elgrandorado,https://www.reddit.com/r/stocks/comments/1j0w7...,7,NaN,NaN,Ticker & %\n\nSPGI: 18.2%\n\nASML: 15.2%\n\nGO...,Ticker & %\n\nSPGI: 18.2%\n\nASML: 15.2%\n\nGO...,0.00000,0.000000,neutral,"[ASML, MA, NTDOY, SPGI, V, GOOG, MANH, Ticker,..."


In [17]:
# --- 5. Save the modified data back to CSV ---
output_file_path = "Sentimented(Neutral).csv"
df.to_csv(output_file_path, index=False)
print(f"Modified data saved to {output_file_path}")

# --- Performance Metrics ---
total_time = preprocessing_time + sentiment_analysis_time + entity_extraction_time
records_per_second = len(df) / total_time  # records classified per second

print(f"\n--- Performance Metrics ---")
print(f"Total Time for Preprocessing, Sentiment Analysis, and Entity Extraction: {total_time:.4f} seconds")
print(f"Records Classified per Second: {records_per_second:.2f} records/second")

Modified data saved to Sentimented(Neutral).csv

--- Performance Metrics ---
Total Time for Preprocessing, Sentiment Analysis, and Entity Extraction: 72.0409 seconds
Records Classified per Second: 140.57 records/second


In [18]:
#Combining the labelled data 
# Load the datasets
df1 = pd.read_csv('/Users/jaredog/Downloads/git code/SC4021-Info-retrieval/ClassificationNew/JupyterNotebook/Sentimented(Neutral).csv')  # Replace with your dataset file paths
df2 = pd.read_csv('/Users/jaredog/Downloads/git code/SC4021-Info-retrieval/ClassificationNew/JupyterNotebook/Labelled.csv')

df1['sentiment'] = df1['sentiment'].replace({'positive': 1.0, 'negative': -1.0})
# Merge the datasets based on 'post_id' column
merged_df = pd.merge(df1, df2[['post_id', ' Label']], on='post_id', how='inner')

# If you want to append the 'label' column from df2 to df1
df1['label'] = merged_df[' Label']
#df1 = df1[df1['label'] != 0.0]
#df1 = df1.dropna(subset=['label']) #changed

# Save the resulting dataframe to a new CSV file
df1.to_csv('Sentimented+Labelled(neutral).csv', index=False)
df1.head()

,type,datetime,post_id,subreddit,title,author,url,upvotes,downvotes,upvote_ratio,body,cleaned_body,subjectivity,polarity,sentiment,entities,label
0,post,2025-03-01 10:00:42,1j0w73o,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,AutoModerator,https://www.reddit.com/r/stocks/comments/1j0w7...,79,4.0,0.95,Please use this thread to discuss your portfol...,Please use this thread to discuss your portfol...,0.36803,0.176667,1.0,"['Losing', 'paper', 'portfolio', '%', '/', 'bo...",0.0
1,comment,2025-03-04 13:54:48,mfykqf0,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,Jimlad73,https://www.reddit.com/r/stocks/comments/1j0w7...,12,NaN,NaN,100% S&P and bricking it,100% S&P and bricking it,0.00000,0.000000,neutral,"['S', 'P', '%', 'S&P']",0.0
2,comment,2025-03-11 00:18:18,mh4b63d,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,inopia,https://www.reddit.com/r/stocks/comments/1j0w7...,11,NaN,NaN,"100% 6-month tbills, lol :)","100% 6-month tbills, lol :)",0.85000,0.650000,1.0,"['%', 'lol']",0.0
3,comment,2025-03-11 21:38:43,mha0zp1,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,thenuttyhazlenut,https://www.reddit.com/r/stocks/comments/1j0w7...,6,NaN,NaN,|Ticker|Company|Allocation|\n|:-|:-|:-|\n|ACGL...,|Ticker|Company|Allocation|\n|:-|:-|:-|\n|ACGL...,0.49250,0.267500,1.0,"['Design|12.75', '|PBR|Petroleo', '|WISE', 'in...",1.0
4,comment,2025-03-13 15:57:56,mhl2a2j,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,elgrandorado,https://www.reddit.com/r/stocks/comments/1j0w7...,7,NaN,NaN,Ticker & %\n\nSPGI: 18.2%\n\nASML: 15.2%\n\nGO...,Ticker & %\n\nSPGI: 18.2%\n\nASML: 15.2%\n\nGO...,0.00000,0.000000,neutral,"['ASML', 'MA', 'NTDOY', 'SPGI', 'V', 'GOOG', '...",0.0


In [20]:
data = df1

# Step 1: Vectorize the cleaned text using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
# make sure no NaNs, and everything is a str
data['cleaned_body'] = data['cleaned_body'].fillna("").astype(str) #added

X_text = vectorizer.fit_transform(data['cleaned_body'])

In [21]:
# Step 2: Add subjectivity and polarity as features
X_features = data[['subjectivity', 'polarity']].values

# Combine the text features and sentiment features
from scipy.sparse import hstack
X_combined = hstack([X_text, X_features])

In [22]:
# Step 4: Convert 'entities' column into binary features (one-hot encoding)
mlb = MultiLabelBinarizer()
entity_features = mlb.fit_transform(data['entities'].apply(eval))  # Convert string lists into actual lists

# Combine entity features with the other features
X_combined_with_entities = hstack([X_combined, entity_features])

# Apply PCA for dimensionality reduction
pca = PCA(n_components=50)
X_combined_with_entities_pca = pca.fit_transform(X_combined_with_entities.toarray())

In [48]:
# Step 5: Apply KMeans clustering
start_time = time.time()

kmeans_with_entities = KMeans(n_clusters=5, random_state=42) #cluster 6 
kmeans_with_entities.fit(X_combined_with_entities_pca)

# Add cluster labels to the dataframe
predicted_labels = kmeans_with_entities.labels_
predicted_labels = predicted_labels.astype(int)

end_time = time.time()
total_time = end_time - start_time
records_classified_per_second = len(data) / total_time  # Assuming 'data' contains all records

# Step 6: Map clusters to sentiment labels
# Create a mapping based on the majority sentiment in each cluster
cluster_sentiment_mapping = {}

for cluster in range(kmeans_with_entities.n_clusters):
    # Find the rows that belong to this cluster
    cluster_rows = data[predicted_labels == cluster]
    # Majority sentiment in the cluster
    majority_sentiment = cluster_rows['label'].mode()[0]
    # Map this cluster to the majority sentiment
    cluster_sentiment_mapping[cluster] = majority_sentiment

# Map predicted labels to sentiment labels
mapped_sentiment_labels = [cluster_sentiment_mapping[label] for label in predicted_labels]

In [49]:
# Step 7: Get the true labels for the remaining rows
true_labels = data['label']
true_labels = np.array(true_labels, dtype=float).astype(int)

# Step 8: Evaluate clustering performance using Precision, Recall, F1-Score
precision = precision_score(true_labels, mapped_sentiment_labels, average='weighted', zero_division=1)  # Handling zero divisions
recall = recall_score(true_labels, mapped_sentiment_labels, average='weighted', zero_division=1)
f1 = f1_score(true_labels, mapped_sentiment_labels, average='weighted', zero_division=1)
accuracy = accuracy_score(true_labels, mapped_sentiment_labels)

# Print evaluation metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Records Classified per Second: {records_classified_per_second:.2f}")

Precision: 0.9532
Recall: 0.9508
F1-Score: 0.9269
Accuracy: 0.9508
Records Classified per Second: 336870.91


/var/folders/bp/974ksd917bx0z_pnb3yqq52h0000gn/T/ipykernel_57364/2961607586.py:3: RuntimeWarning: invalid value encountered in cast
  true_labels = np.array(true_labels, dtype=float).astype(int)


In [50]:
# 1. Calculate Centroid Distances (Euclidean distance between centroids)
centroids = kmeans_with_entities.cluster_centers_
inter_cluster_distances = np.linalg.norm(centroids[:, np.newaxis] - centroids, axis=2)
print("Inter-cluster centroid distances:\n", inter_cluster_distances)

# 2. Calculate Silhouette Score
silhouette_avg = silhouette_score(X_combined_with_entities_pca, predicted_labels)
print(f"Silhouette Score: {silhouette_avg:.4f}")

# 3. Calculate Davies-Bouldin Index
davies_bouldin = davies_bouldin_score(X_combined_with_entities_pca, predicted_labels)
print(f"Davies-Bouldin Index: {davies_bouldin:.4f}")

# 4. Calculate Average Intra-cluster Similarity (average distance within clusters)
# Intra-cluster distance (average distance within each cluster)
intra_cluster_similarity = []
for cluster_id in range(kmeans_with_entities.n_clusters):
    cluster_points = X_combined_with_entities_pca[predicted_labels == cluster_id]
    cluster_center = centroids[cluster_id]
    intra_cluster_similarity.append(np.mean(np.linalg.norm(cluster_points - cluster_center, axis=1)))

average_intra_cluster_similarity = np.mean(intra_cluster_similarity)
print(f"Average Intra-cluster Similarity: {average_intra_cluster_similarity:.4f}")

Inter-cluster centroid distances:
 [[0.         0.84018304 1.08723436 0.60411424 2.03532774]
 [0.84018304 0.         1.19097772 0.91319153 1.99876547]
 [1.08723436 1.19097772 0.         1.16667745 2.24825678]
 [0.60411424 0.91319153 1.16667745 0.         2.08940818]
 [2.03532774 1.99876547 2.24825678 2.08940818 0.        ]]
Silhouette Score: 0.1204
Davies-Bouldin Index: 2.6550
Average Intra-cluster Similarity: 1.6433


In [32]:
#random test
random_labels = np.random.choice([1, -1], size=len(data), replace=True)

# Step 2: Get the predicted labels from your trained KMeans model
predicted_labels = kmeans_with_entities.labels_

# Step 3: Evaluate the clustering performance by comparing predicted labels to random labels
accuracy = accuracy_score(random_labels, predicted_labels)
precision = precision_score(random_labels, predicted_labels, average='weighted', zero_division=1)
recall = recall_score(random_labels, predicted_labels, average='weighted', zero_division=1)
f1 = f1_score(random_labels, predicted_labels, average='weighted', zero_division=1)

# Step 4: Print the evaluation metrics
print(f"Random Accuracy Test Results:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Random Accuracy Test Results:
Accuracy: 0.3906
Precision: 0.7497
Recall: 0.3906
F1-Score: 0.2865


In [58]:
from transformers import BertTokenizer, BertModel
import torchfrom transformers 
import BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset


In [42]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings for a given text
def get_bert_embeddings(texts):
    embeddings = []
    for text in texts:
        # Tokenize and encode the text
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        
        # Get the embeddings from BERT (the output of the last hidden layer)
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Extract the last hidden state (embeddings)
        last_hidden_states = outputs.last_hidden_state
        # Get the mean of all token embeddings to get the sentence embedding
        sentence_embedding = torch.mean(last_hidden_states, dim=1).squeeze().numpy()
        embeddings.append(sentence_embedding)
    
    return np.array(embeddings)

# Example usage to get BERT embeddings for your dataset
texts = data['cleaned_body'].tolist()  # Assuming you have a column 'cleaned_body'
bert_embeddings = get_bert_embeddings(texts)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [43]:
pca = PCA(n_components=50)
bert_embeddings_reduced = pca.fit_transform(bert_embeddings)

# Step 5: Apply KMeans clustering using BERT embeddings
kmeans_with_bert = KMeans(n_clusters=6, random_state=42)
kmeans_with_bert.fit(bert_embeddings_reduced)

# Add cluster labels to the dataframe
predicted_labels_bert = kmeans_with_bert.labels_

In [56]:
# Prepare the dataset for fine-tuning

dataset = Dataset.from_pandas(data)  # Assuming your data is in a Pandas DataFrame
dataset = dataset.map(lambda x: tokenizer(x['cleaned_body'], truncation=True, padding=True, max_length=512))

# Fine-tune BERT for sentiment analysis
model_for_sentiment = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 labels (positive/negative)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=200,
)

trainer = Trainer(
    model=model_for_sentiment,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,  # Or use a separate validation dataset
)

trainer.train()

Map: 100%|███████████████████████████| 361/361 [00:00<00:00, 2023.24 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [55]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS backend")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using MPS backend


In [57]:
import torch
print(torch.__version__)

2.6.0
